In [1]:
import sys
sys.path.append("../src")
from dataio import load_dam_rivout, load_dam_discharge


### 1. Daily Evaluation

##### 1.1 全体なエラー

- Use the different metrics you have and present them in one clear plot
- Compare with linear model

##### 1.２ ダムごとの分析

- ダムごとのエラーの分布
- うまく行ってるダムの共有点ありますか？　（ダムのサイズ、県など）
- うまく行ってないダムの共有点ありますか？

##### 1.3 季節の影響

- 季節によってエラーはよく変わりますか？季節は大きく影響ありますか？
- 月毎のエラープロット
- 月の影響：error = f(雪とける)

### 2. Hourly Evaluation